In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from datetime import date, timedelta

def read_latlong():
    return pd.read_csv('latlong.csv')

def getData(start_date,end_date, latlong_df):
    main_df = []
    for single_date in daterange(start_date, end_date):
            currentdate = single_date.strftime("%Y-%m-%d")
            for (colname,colval) in latlong_df.iterrows():
                generated_url = generateUrl(colval.values[1], colval.values[2], currentdate)
                indvidual_df = writetocsv(generated_url, colval.values[0], currentdate)
                main_df.append(indvidual_df)
    return main_df

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

def generateUrl(latitude,longitude,date):
    return f'https://api.brightsky.dev/weather?lat={latitude}&lon={longitude}&date={date}'

             
def writetocsv(url ,districtName, currentdate):
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    json_dicti = json.loads(soup.text)
    tocsv = pd.DataFrame(json_dicti['weather'])
    tocsv.insert(0, 'District Name', districtName)
    tocsv.insert(1, 'Current Date', currentdate)
    return tocsv

def write_to_csv(main_dataframe):
    with open("1yearDataUpdated.csv", "a", newline="") as csvfile:
        count = 0
        print("Writing to file initiated")
        for each_df in main_dataframe:
            if count == 0:
                each_df.to_csv(csvfile,index=False)
                count = 1
            else:
                each_df.to_csv(csvfile,header=False,index=False)
    print("Writing to file Finished")
    

In [6]:
start_date = date(2022, 5, 22)  #year-month-date format
end_date = date(2022, 8, 1)
latlong_Dataframe = read_latlong()
main_dataframe = getData(start_date,end_date, latlong_Dataframe)
write_to_csv(main_dataframe)

Writing to file initiated
Writing to file Finished
